In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import xarray as xr
import cftime
from dateutil.relativedelta import relativedelta
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from math import sin, cos, sqrt, atan2, radians
from scipy import stats
from sklearn.metrics import mean_squared_error

In [2]:
#importing data

indices = pd.read_csv("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/indices.csv")
narr = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/NARR.nc")
E1 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E1.nc")
E2 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E2.nc")
E3 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E3.nc")
E4 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E4.nc")
E5 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E5.nc")
E6 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E6.nc")
E7 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E7.nc")
E8 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E8.nc")
E9 = xr.open_dataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E9.nc")

In [4]:
min_lon = 0
min_lat = 0
max_lon = 360
max_lat = 90

mask_lon = (E1.lon >= min_lon) & (E1.lon <= max_lon)
mask_lat = (E1.lat >= min_lat) & (E1.lat <= max_lat)

cropped_E1 = E1.where(mask_lon & mask_lat, drop=True)
cropped_E2 = E2.where(mask_lon & mask_lat, drop=True)
cropped_E3 = E3.where(mask_lon & mask_lat, drop=True)
cropped_E4 = E4.where(mask_lon & mask_lat, drop=True)
cropped_E5 = E5.where(mask_lon & mask_lat, drop=True)
cropped_E6 = E6.where(mask_lon & mask_lat, drop=True)
cropped_E7 = E7.where(mask_lon & mask_lat, drop=True)
cropped_E8 = E8.where(mask_lon & mask_lat, drop=True)
cropped_E9 = E9.where(mask_lon & mask_lat, drop=True)

In [8]:
cropped_E1.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E1.nc4")
cropped_E2.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E2.nc4")
cropped_E3.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E3.nc4")
cropped_E4.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E4.nc4")
cropped_E5.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E5.nc4")
cropped_E6.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E6.nc4")
cropped_E7.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E7.nc4")
cropped_E8.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E8.nc4")
cropped_E9.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/cropped_E9.nc4")

In [63]:
E1.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E1.nc4")
E2.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E2.nc4")
E3.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E3.nc4")
E4.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E4.nc4")
E5.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E5.nc4")
E6.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E6.nc4")
E7.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E7.nc4")
E8.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E8.nc4")
E9.to_netcdf("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/E9.nc4")

In [165]:
nlat = narr.lat
nlon = narr.lon
elat = cropped_E1.lat
elon = cropped_E1.lon

n_acpcp = narr.acpcp
e1_acpcp = cropped_E1.PRECT
e2_acpcp = cropped_E2.PRECT
e3_acpcp = cropped_E3.PRECT
e4_acpcp = cropped_E4.PRECT
e5_acpcp = cropped_E5.PRECT
e6_acpcp = cropped_E6.PRECT
e7_acpcp = cropped_E7.PRECT
e8_acpcp = cropped_E8.PRECT
e9_acpcp = cropped_E9.PRECT

z = np.zeros(len(elat)*len(elon),dtype="int")

e1_nlon, e1_nlat, e1_lon, e1_lat, e1_distance, e1_ks_stat, e1_ks_pval, e1_rmse = ([] for i in range(8))
e2_nlon, e2_nlat, e2_lon, e2_lat, e2_distance, e2_ks_stat, e2_ks_pval, e2_rmse = ([] for i in range(8))
e3_nlon, e3_nlat, e3_lon, e3_lat, e3_distance, e3_ks_stat, e3_ks_pval, e3_rmse = ([] for i in range(8))
e4_nlon, e4_nlat, e4_lon, e4_lat, e4_distance, e4_ks_stat, e4_ks_pval, e4_rmse = ([] for i in range(8))
e5_nlon, e5_nlat, e5_lon, e5_lat, e5_distance, e5_ks_stat, e5_ks_pval, e5_rmse = ([] for i in range(8))
e6_nlon, e6_nlat, e6_lon, e6_lat, e6_distance, e6_ks_stat, e6_ks_pval, e6_rmse = ([] for i in range(8))
e7_nlon, e7_nlat, e7_lon, e7_lat, e7_distance, e7_ks_stat, e7_ks_pval, e7_rmse = ([] for i in range(8))
e8_nlon, e8_nlat, e8_lon, e8_lat, e8_distance, e8_ks_stat, e8_ks_pval, e8_rmse = ([] for i in range(8))
e9_nlon, e9_nlat, e9_lon, e9_lat, e9_distance, e9_ks_stat, e9_ks_pval, e9_rmse = ([] for i in range(8))


In [166]:
def point_distance(lat1,lon1,lat2,lon2):
    
    R = 6373.0
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = 1000*R * c
    
    return distance

In [181]:
for i in range(len(elon)):
    for j in range(len(elat)):
        print(i+1,j+1)
        min_distance = 100000000000000000000
        min_lon = np.nan
        min_lat = np.nan
        current_lon = elon[i].values-180
        current_lat = elat[j].values
        stopper = 0
        distance = 0
        
        for k in range(len(nlon)):
            
            if stopper == 1:
                
                break
                
            for l in range(len(nlon[1])):
                
                distance = point_distance(elat[j].values,elon[i].values-180,nlat[k,l].values,nlon[k,l].values)
                
                if distance < min_distance:
                    
                    min_distance = distance
                    min_lon = nlon[k,l].values.item()
                    min_lat = nlat[k,l].values.item()
                    
                    if np.isnan(list(n_acpcp[:,k,l].values)).all():
                        
                        ks_vals_e1 = [np.nan,np.nan]
                        ks_vals_e2 = [np.nan,np.nan]
                        ks_vals_e3 = [np.nan,np.nan]
                        ks_vals_e4 = [np.nan,np.nan]
                        ks_vals_e5 = [np.nan,np.nan]
                        ks_vals_e6 = [np.nan,np.nan]
                        ks_vals_e7 = [np.nan,np.nan]
                        ks_vals_e8 = [np.nan,np.nan]
                        ks_vals_e9 = [np.nan,np.nan]
                        e1_rmse_val = np.nan
                        e2_rmse_val = np.nan
                        e3_rmse_val = np.nan
                        e4_rmse_val = np.nan
                        e5_rmse_val = np.nan
                        e6_rmse_val = np.nan
                        e7_rmse_val = np.nan
                        e8_rmse_val = np.nan
                        e9_rmse_val = np.nan
                        
                    else:
                        
                        ks_vals_e1 = stats.ks_2samp(n_acpcp[:,k,l].values,e1_acpcp[:,j,i].values)
                        ks_vals_e2 = stats.ks_2samp(n_acpcp[:,k,l].values,e2_acpcp[:,j,i].values)
                        ks_vals_e3 = stats.ks_2samp(n_acpcp[:,k,l].values,e3_acpcp[:,j,i].values)
                        ks_vals_e4 = stats.ks_2samp(n_acpcp[:,k,l].values,e4_acpcp[:,j,i].values)
                        ks_vals_e5 = stats.ks_2samp(n_acpcp[:,k,l].values,e5_acpcp[:,j,i].values)
                        ks_vals_e6 = stats.ks_2samp(n_acpcp[:,k,l].values,e6_acpcp[:,j,i].values)
                        ks_vals_e7 = stats.ks_2samp(n_acpcp[:,k,l].values,e7_acpcp[:,j,i].values)
                        ks_vals_e8 = stats.ks_2samp(n_acpcp[:,k,l].values,e8_acpcp[:,j,i].values)
                        ks_vals_e9 = stats.ks_2samp(n_acpcp[:,k,l].values,e9_acpcp[:,j,i].values)
                        
                        e1_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e1_acpcp':e1_acpcp[:,j,i].values})
                        e2_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e2_acpcp':e2_acpcp[:,j,i].values})
                        e3_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e3_acpcp':e3_acpcp[:,j,i].values})
                        e4_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e4_acpcp':e4_acpcp[:,j,i].values})
                        e5_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e5_acpcp':e5_acpcp[:,j,i].values})
                        e6_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e6_acpcp':e6_acpcp[:,j,i].values})
                        e7_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e7_acpcp':e7_acpcp[:,j,i].values})
                        e8_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e8_acpcp':e8_acpcp[:,j,i].values})
                        e9_df = pd.DataFrame({'n_acpcp':n_acpcp[:,k,l].values,'e9_acpcp':e9_acpcp[:,j,i].values})
                        
                        e1_df = e1_df.dropna()
                        e2_df = e2_df.dropna()
                        e3_df = e3_df.dropna()
                        e4_df = e4_df.dropna()
                        e5_df = e5_df.dropna()
                        e6_df = e6_df.dropna()
                        e7_df = e7_df.dropna()
                        e8_df = e8_df.dropna()
                        e9_df = e9_df.dropna()
                        
                        e1_rmse_val = sqrt(mean_squared_error(e1_df['n_acpcp'],e1_df['e1_acpcp']))
                        e2_rmse_val = sqrt(mean_squared_error(e2_df['n_acpcp'],e2_df['e2_acpcp']))
                        e3_rmse_val = sqrt(mean_squared_error(e3_df['n_acpcp'],e3_df['e3_acpcp']))
                        e4_rmse_val = sqrt(mean_squared_error(e4_df['n_acpcp'],e4_df['e4_acpcp']))
                        e5_rmse_val = sqrt(mean_squared_error(e5_df['n_acpcp'],e5_df['e5_acpcp']))
                        e6_rmse_val = sqrt(mean_squared_error(e6_df['n_acpcp'],e6_df['e6_acpcp']))
                        e7_rmse_val = sqrt(mean_squared_error(e7_df['n_acpcp'],e7_df['e7_acpcp']))
                        e8_rmse_val = sqrt(mean_squared_error(e8_df['n_acpcp'],e8_df['e8_acpcp']))
                        e9_rmse_val = sqrt(mean_squared_error(e9_df['n_acpcp'],e9_df['e9_acpcp']))
                else:
                    
                    stopper = 1
        
        e1_lon.append(current_lon)
        e1_lat.append(current_lat)
        e1_distance.append(min_distance)
        e1_nlon.append(min_lon)
        e1_nlat.append(min_lat)
        e1_ks_stat.append(ks_vals_e1[0])
        e1_ks_pval.append(ks_vals_e1[1])
        e1_rmse.append(e1_rmse_val)
        
        e2_lon.append(current_lon)
        e2_lat.append(current_lat)
        e2_distance.append(min_distance)
        e2_nlon.append(min_lon)
        e2_nlat.append(min_lat)
        e2_ks_stat.append(ks_vals_e2[0])
        e2_ks_pval.append(ks_vals_e2[1])
        e2_rmse.append(e2_rmse_val)
        
        e3_lon.append(current_lon)
        e3_lat.append(current_lat)
        e3_distance.append(min_distance)
        e3_nlon.append(min_lon)
        e3_nlat.append(min_lat)
        e3_ks_stat.append(ks_vals_e3[0])
        e3_ks_pval.append(ks_vals_e3[1])
        e3_rmse.append(e3_rmse_val)
        
        e4_lon.append(current_lon)
        e4_lat.append(current_lat)
        e4_distance.append(min_distance)
        e4_nlon.append(min_lon)
        e4_nlat.append(min_lat)
        e4_ks_stat.append(ks_vals_e4[0])
        e4_ks_pval.append(ks_vals_e4[1])
        e4_rmse.append(e4_rmse_val)
        
        e5_lon.append(current_lon)
        e5_lat.append(current_lat)
        e5_distance.append(min_distance)
        e5_nlon.append(min_lon)
        e5_nlat.append(min_lat)
        e5_ks_stat.append(ks_vals_e5[0])
        e5_ks_pval.append(ks_vals_e5[1])
        e5_rmse.append(e5_rmse_val)
        
        e6_lon.append(current_lon)
        e6_lat.append(current_lat)
        e6_distance.append(min_distance)
        e6_nlon.append(min_lon)
        e6_nlat.append(min_lat)
        e6_ks_stat.append(ks_vals_e6[0])
        e6_ks_pval.append(ks_vals_e6[1])
        e6_rmse.append(e6_rmse_val)
        
        e7_lon.append(current_lon)
        e7_lat.append(current_lat)
        e7_distance.append(min_distance)
        e7_nlon.append(min_lon)
        e7_nlat.append(min_lat)
        e7_ks_stat.append(ks_vals_e7[0])
        e7_ks_pval.append(ks_vals_e7[1])
        e7_rmse.append(e7_rmse_val)
        
        e8_lon.append(current_lon)
        e8_lat.append(current_lat)
        e8_distance.append(min_distance)
        e8_nlon.append(min_lon)
        e8_nlat.append(min_lat)
        e8_ks_stat.append(ks_vals_e8[0])
        e8_ks_pval.append(ks_vals_e8[1])
        e8_rmse.append(e8_rmse_val)
        
        e9_lon.append(current_lon)
        e9_lat.append(current_lat)
        e9_distance.append(min_distance)
        e9_nlon.append(min_lon)
        e9_nlat.append(min_lat)
        e9_ks_stat.append(ks_vals_e9[0])
        e9_ks_pval.append(ks_vals_e9[1])
        e9_rmse.append(e9_rmse_val)


1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 68
2 69
2 70
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 88
2 89
2 90
2 91
2 92
2 93
2 94
2 95
2 96
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 

In [185]:
e1_distance_df = pd.DataFrame({'e1_lon':e1_lon,'e1_lat':e1_lat,'narr_lon':e1_nlon,'narr_lat':e1_nlat,'distance':e1_distance,'stat':e1_ks_stat,'p-value':e1_ks_pval,'rmse':e1_rmse})
e2_distance_df = pd.DataFrame({'e2_lon':e2_lon,'e2_lat':e2_lat,'narr_lon':e2_nlon,'narr_lat':e2_nlat,'distance':e2_distance,'stat':e2_ks_stat,'p-value':e2_ks_pval,'rmse':e2_rmse})
e3_distance_df = pd.DataFrame({'e3_lon':e3_lon,'e3_lat':e3_lat,'narr_lon':e3_nlon,'narr_lat':e3_nlat,'distance':e3_distance,'stat':e3_ks_stat,'p-value':e3_ks_pval,'rmse':e3_rmse})
e4_distance_df = pd.DataFrame({'e4_lon':e4_lon,'e4_lat':e4_lat,'narr_lon':e4_nlon,'narr_lat':e4_nlat,'distance':e4_distance,'stat':e4_ks_stat,'p-value':e4_ks_pval,'rmse':e4_rmse})
e5_distance_df = pd.DataFrame({'e5_lon':e5_lon,'e5_lat':e5_lat,'narr_lon':e5_nlon,'narr_lat':e5_nlat,'distance':e5_distance,'stat':e5_ks_stat,'p-value':e5_ks_pval,'rmse':e5_rmse})
e6_distance_df = pd.DataFrame({'e6_lon':e6_lon,'e6_lat':e6_lat,'narr_lon':e6_nlon,'narr_lat':e6_nlat,'distance':e6_distance,'stat':e6_ks_stat,'p-value':e6_ks_pval,'rmse':e6_rmse})
e7_distance_df = pd.DataFrame({'e7_lon':e7_lon,'e7_lat':e7_lat,'narr_lon':e7_nlon,'narr_lat':e7_nlat,'distance':e7_distance,'stat':e7_ks_stat,'p-value':e7_ks_pval,'rmse':e7_rmse})
e8_distance_df = pd.DataFrame({'e8_lon':e8_lon,'e8_lat':e8_lat,'narr_lon':e8_nlon,'narr_lat':e8_nlat,'distance':e8_distance,'stat':e8_ks_stat,'p-value':e8_ks_pval,'rmse':e8_rmse})
e9_distance_df = pd.DataFrame({'e9_lon':e9_lon,'e9_lat':e9_lat,'narr_lon':e9_nlon,'narr_lat':e9_nlat,'distance':e9_distance,'stat':e9_ks_stat,'p-value':e9_ks_pval,'rmse':e9_rmse})

In [186]:
e1_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e1_ks_results.csv')
e2_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e2_ks_results.csv')
e3_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e3_ks_results.csv')
e4_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e4_ks_results.csv')
e5_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e5_ks_results.csv')
e6_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e6_ks_results.csv')
e7_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e7_ks_results.csv')
e8_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e8_ks_results.csv')
e9_distance_df.to_csv('/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/climate_model_data/e9_ks_results.csv')